In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
# os.chdir('/content/drive/My Drive/metall/')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit



path = '/content/drive/MyDrive/Прогноз металлические заготовки/'
file_name = 'data_for_lstm.csv'

df = pd.read_csv(path + file_name)


## LSTM

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

def lstm_trained(features_set,labels,epochs = 40, batch_size = 4,num_layers=1,p=0.0,units=48,optimizer='adam'):
  '''
  
  return trained lstm

  '''
  model = Sequential()
  if num_layers==2:
    model.add(LSTM(units=units, return_sequences=True, input_shape=(features_set.shape[1],features_set.shape[2] )))
    model.add(Dropout(p))
    
  elif num_layers>2:
    model.add(LSTM(units=units, return_sequences=True, input_shape=(features_set.shape[1],features_set.shape[2] )))
    for layer in range(num_layers-1):
      model.add(LSTM(units=units, return_sequences=True))
      model.add(Dropout(p))

  model.add(LSTM(units=50))
  model.add(Dropout(p))
  model.add(Dense(units = 1))
  model.compile(optimizer = optimizer, loss = 'mean_squared_error')
  model.fit(features_set, labels, epochs = epochs, batch_size = batch_size, verbose=1)
  return model



## lstm score

In [ ]:
def lstm_predict(df_train,df_test,shift, epochs = 2, batch_size = 8,num_layers=1,p=0.0,units=48,optimizer='adam'):
  '''

  returns the prediction lstm

  params:
  shift - the length of the background
  other - dataframes or params for lstm

  '''
  scaler = MinMaxScaler(feature_range = (0, 1))
  df_training_processed = df_train.iloc[:, :].values
  df_training_scaled = scaler.fit_transform(df_training_processed)

  features_set = []
  labels = []
  # We will make predictions for the next week based on the history (length of shift=length_history)
  # Let's form this selection
  len_ = df_training_scaled.shape[0]
  for i in range(shift, len_):
      features_set.append(df_training_scaled[i-shift:i, :])
      labels.append(df_training_scaled[i, 0])

  # Convert to an np array and make a suitable format for LSTM, train the model
  features_set, labels = np.array(features_set), np.array(labels)
  features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], features_set.shape[2]))
  model = lstm_trained(features_set, labels,epochs=epochs,batch_size=batch_size, p=p, num_layers=num_layers, optimizer=optimizer, units=units)

  # Let's create a sample for testing
  df_total = pd.concat((df_train, df_test), axis=0)
  test_inputs = df_total[len(df_total) - len(df_test) - shift:]
  test_inputs_processed = test_inputs.iloc[:, :].values
  test_inputs_scaled= scaler.transform(test_inputs_processed)
  test_features = []
  for i in range(shift, len(test_inputs_scaled)):
      test_features.append(test_inputs_scaled[i-shift:i, :])

  # Convert again to an np array and make a format suitable for LSTM  test_features = np.array(test_features)
  test_features = np.reshape(test_features, (test_features.shape[0], test_features.shape[1], test_features.shape[2]))

  predictions = model.predict(test_features)
  predictions=predictions.reshape(df_test.shape[0],)

  test_data_processed = (df_test.copy()).iloc[:].values
  test_data_processed[:,0] = predictions
  test_data_processed = scaler.inverse_transform(test_data_processed)
  predictions = test_data_processed[:,0]

  return predictions

## Grid



In [ ]:
from sklearn.model_selection import ParameterGrid

grid = ParameterGrid({'num_layers': [0,1], 
                      'epochs':[40,60],
                      'p':[0.0,0.05,0.1],
                      'length_history':[4,8],
                      'n_splits':[10,20],
                      'units':[8,12,24,48],
                      'min_len_datasets':[30],
                      'length_prediction':[1]})

In [ ]:
#grid = ParameterGrid({'num_layers': [1], 
#                      'epochs':[40],
#                      'p':[0.2],
#                      'length_history':[8],
#                      'n_splits':[10],
#                      'min_len_datasets':[30],
#                      'length_prediction':[1]})

let's calculate the average mae for different grid architectures using timeseriescv




In [ ]:
from tqdm.notebook import tqdm

scores = dict()

for item in tqdm(grid):  
  values = (df.copy()).values
  length_prediction = item['length_prediction']
  n_splits = item['n_splits']
  min_len_datasets= item['min_len_datasets']
  length_history=item['length_history']
  tscv = TimeSeriesSplit(n_splits=n_splits,test_size=length_prediction)
  res = []
  for id_train,id_test in tscv.split(values):
    df_train=df.iloc[id_train,:].copy()
    if (len(df_train)<length_history+min_len_datasets):
      continue
    df_test=df.iloc[id_test,:].copy()
    predictions = lstm_predict(df_train.copy(), df_test.copy(), shift=length_history,
                               epochs=item['epochs'],p=item['p'],num_layers=item['num_layers'],units=item['units'])
    error = 0
    targets = df_test['target'].values
    for i in range(len(predictions)):
      error+=abs(predictions[i]-targets[i])
    res.append(error/len(predictions))
    print(scores)

  scores[str(item)] = [np.mean(res)] 

In [ ]:
scores